In [ ]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
""" pdf_loader = DirectoryLoader('../../training_data/', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../training_data/', glob="**/*.txt")
word_loader = DirectoryLoader('../../training_data/', glob="**/*.docx")
csv_loader = DirectoryLoader('../../training_data/', glob="**/*.csv") """

In [ ]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/cms/hubl"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=200, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = Chroma.from_documents(texts, embeddings, collection_name="hubspot_hubl_knowledge_base")

In [ ]:
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate
)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain.llms import Cohere
cohere_api_key= os.getenv('COHERE_API_KEY')
cohere_llm = ChatLiteLLM(model="command-nightly")


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
google_api_key= os.getenv('GOOGLE_API_KEY')
chat = ChatLiteLLM(model="palm/chat-bison")

In [ ]:
hubl_docs_chain = RetrievalQA.from_chain_type(
    llm=cohere_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [ ]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [ ]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()


#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

In [ ]:
tools = [
    Tool(
        name="HubL Docs",
        func=hubl_docs_chain.run,
        description="Useful for when you need to answer questions about HubL. output should be code",
    ),
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, cohere_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "What is HubL?"
)